In [ ]:
import re
from textnode import (
    TextNode,
    text_type_text,
    text_type_bold,
    text_type_italic,
    text_type_code,
    text_type_image,
    text_type_link,
)


def split_nodes_delimiter(old_nodes, delimiter, text_type):
    new_nodes = []
    for old_node in old_nodes:
        if old_node.text_type not in [
            text_type_text,
            text_type_bold,
            text_type_italic,
            text_type_code,
            text_type_link,
            text_type_image,
        ]:
            new_nodes.append(old_node)
            continue
        if old_node.text_type == text_type_image:
            new_nodes.extend(split_nodes_image(old_node.text))
        elif old_node.text_type == text_type_link:
            new_nodes.extend(split_nodes_link(old_node.text))
        else:
            split_nodes = []
            sections = old_node.text.split(delimiter)
            if len(sections) % 2 == 0:
                raise ValueError("Invalid markdown, formatted section not closed")
            for i in range(len(sections)):
                if sections[i] == "":
                    continue
                if i % 2 == 0:
                    split_nodes.append(TextNode(sections[i], text_type_text))
                else:
                    split_nodes.append(TextNode(sections[i], text_type))
            new_nodes.extend(split_nodes)
    return new_nodes


def split_nodes_image(old_nodes, text):
    nodes = []
    image_pattern = r"!\[(.*?)\]\((.*?)\)"
    for match in re.finditer(image_pattern, text):
        image_tup = match.groups()
        before, after = text.split(f"![{image_tup[0]}]({image_tup[1]})", 1)
        if before:
            nodes.extend(old_nodes(before, text_type_text))
        nodes.append(TextNode(f"![{image_tup[0]}]({image_tup[1]})", text_type_image))
        text = after
    if text:
        nodes.extend(old_nodes(text, text_type_text))
    return nodes


def split_nodes_link(old_nodes, text):
    nodes = []
    link_pattern = r"\[(.*?)\]\((.*?)\)"
    for match in re.finditer(link_pattern, text):
        link_tup = match.groups()
        before, after = text.split(f"[{link_tup[0]}]({link_tup[1]})", 1)
        if before:
            nodes.extend(old_nodes(before, text_type_text))
        nodes.append(TextNode(f"[{link_tup[0]}]({link_tup[1]})", text_type_link))
        text = after
    if text:
        nodes.extend(old_nodes(text, text_type_text))
    return nodes